<img width="8%" alt="GitHub.png" src="https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/.github/assets/logos/GitHub.png" style="border-radius: 15%">

# GitHub - Get interactions from closed issues
<a href="https://bit.ly/3JyWIk6">Give Feedback</a> | <a href="https://github.com/jupyter-naas/awesome-notebooks/issues/new?assignees=&labels=bug&template=bug_report.md&title=GitHub+-+Get+interactions+from+closed+issues:+Error+short+description">Bug report</a>

**Tags:** #github #closedIssues #reaction #comment #creator #assignees #automation

**Author:** [Varsha Kumar](https://www.linkedin.com/in/varsha-kumar-590466305/)

**Last update:** 2024-06-10 (Created: 2024-06-10)

**Description:** This notebook allows users to retrieve all comments and reactions from closed issues along with creator and assignee data on a given repository.

## Input

In [1]:
import requests
import re
import pandas as pd
import naas_python as naas
from urllib.parse import urlparse
import time

### Setup variables
- `github_token`: personal token creates
- `github_url`: link to the chosen github repo
- `state`: type of issue

In [2]:
github_token = naas.secret.get("GITHUB_TOKEN").value
repo_url = "https://github.com/jupyter-naas/awesome-notebooks"
state = 'closed'

## Model

### Filter on closed issues without pull requests

In [10]:
def get_filtered_closed_issues(github_url, personal_access_token):
    # Parse the GitHub URL to get the owner and repo name
    parsed_url = urlparse(github_url)
    path_parts = parsed_url.path.strip('/').split('/')
    
    if len(path_parts) != 2:
        raise ValueError("Invalid GitHub URL. It should be in the format: https://github.com/owner/repo")
    
    owner, repo = path_parts
    
    # GitHub API URL for issues
    api_url = f"https://api.github.com/repos/{owner}/{repo}/issues"
    
    # Set up headers with personal access token for authentication
    headers = {
        'Authorization': f'token {personal_access_token}',
        'Accept': 'application/vnd.github.v3+json'
    }
    
    # Set up parameters to get only open issues
    params = {
        'state': state,
        'filter': 'all',
        'per_page': 100  # Set the number of issues per page to 100 (maximum allowed by GitHub API)
    }
    
    # Initialize an empty list to store all open issues excluding pull requests
    filtered_issues = []
    
    # Pagination to fetch all issues
    page = 1
    while True:
        params['page'] = page
        response = requests.get(api_url, headers=headers, params=params)
        
        if response.status_code != 200:
            raise Exception(f"Error fetching issues from GitHub: {response.status_code} {response.reason}")
        
        issues = response.json()
        if not issues:
            break
        
        # Filter out pull requests from the issues
        for issue in issues:
            if 'pull_request' not in issue:
                filtered_issues.append(issue)
        
        page += 1
    
    return filtered_issues


filtered_issues = get_filtered_closed_issues(repo_url, github_token)

### Get interactions from issues

In [11]:
# Get comments and reactions from issues
def get_all_comments(api_url, personal_access_token):
    # Set up headers with personal access token for authentication
    headers = {
        'Authorization': f'token {personal_access_token}',
        'Accept': 'application/vnd.github.v3+json'
    }

    # Get comments for the issue
    response = requests.get(api_url + '/comments', headers=headers)
    if response.status_code != 200:
        raise Exception(f"Error fetching comments from GitHub: {response.status_code} {response.reason}")
    comments = response.json()

    return comments

def get_comment_reactions(comment_url, personal_access_token):
    headers = {
        "Authorization": f"token {personal_access_token}",
        "Accept": "application/vnd.github.squirrel-girl-preview+json"
    }
    # Get reactions for the issue
    response = requests.get(comment_url + "/reactions", headers=headers)
    response.raise_for_status()
    return response.json()
    
def get_interactions(
    url,
    uid,
    interaction_type,
    content,
    time,
    user_login,
    user_id,
):
    return {
        "URL": url,
        "ID": uid,
        "TYPE": interaction_type,
        "CONTENT": content,
        "DATE_TIME": time,
        "USER_LOGIN": user_login,
        "USER_ID": user_id,
    }

## Output
### Display result

In [12]:
data = []
    
for issue in filtered_issues:
    # Creator
    issue_number = issue['number']
    issue_url = issue['url']
    issue_content = issue['title']
    issue_time = issue['created_at']
    issue_user_login = issue['user']['login']
    issue_user_id = issue['user']['id']

    # Get creator
    data_creator = get_interactions(
        issue_url,
        str(issue_number) + "_" +  str(issue_user_id),
        "CREATOR",
        issue_content,
        issue_time,
        issue_user_login,
        issue_user_id,
    )
    data.append(data_creator)
    
    # Get Assignees
    if len(issue['assignees']) > 0:
        for assignee in issue['assignees']:
            data_assignee = get_interactions(
                issue_url,
                str(issue_number) + "_" + str(assignee['id']),
                "ASSIGNEE",
                issue_content,
                issue_time,
                assignee['login'],
                assignee['id'],
            )
            data.append(data_assignee)
            
    # Get comments
    comments = get_all_comments(issue_url, github_token)
    for comment in comments:
        comment_id = comment['id']
        comment_body = comment['body']
        comment_created_at = comment['created_at']
        comment_user_login = comment['user']['login']
        comment_user_id = comment['user']['id']
        data_comment = get_interactions(
            issue_url,
            comment_id,
            "COMMENT",
            comment_body,
            comment_created_at,
            comment_user_login,
            comment_user_id,
        )
        
        data.append(data_comment)
        #print (comment)
        reactions = get_comment_reactions(comment['url'], github_token)
        if len(reactions) > 0:
            for reaction in reactions:
                reaction_id = reaction['id']
                reaction_body = reaction['content']
                reaction_created_at = reaction['created_at']
                reaction_user_login = reaction['user']['login']
                reaction_user_id = reaction['user']['id']
                data_reaction = get_interactions(
                    issue_url,
                    reaction_id,
                    "REACTION",
                    reaction_body,
                    reaction_created_at,
                    reaction_user_login,
                    reaction_user_id,
                )
                data.append(data_reaction)
pd.DataFrame(data)